In [4]:
print("OK")

OK


In [51]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone import Pinecone
from dotenv import load_dotenv
import os
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA




In [6]:
from flask import Flask, render_template, jsonify, request
from src.helper import download_hugging_face_embeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
from src.prompt import *
import os

In [44]:
import os 
import time
import pandas as pd
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

In [45]:
load_dotenv()
api_key = os.getenv('PINECONE_API_KEY')


In [46]:
api_key

'3c87c5b8-95ab-41eb-8afc-e63ce42e96fa'

In [47]:
#Extract PDF Data
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob='*.pdf',
                    loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [48]:
extracted_data=load_pdf("../data/")

In [12]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [13]:
text_chunks = text_split(extracted_data)

In [14]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

/opt/anaconda3/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/opt/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [41]:
querey = embeddings.embed_query("Hello world")
print("Length", len(querey))

Length 384


In [52]:
pc = Pinecone(api_key=api_key)
spec = ServerlessSpec("aws","us-east-1")
index_name="medical-chatbot"

In [53]:
docsearch = PineconeVectorStore.from_existing_index(index_name,embeddings)
querey = "What are allergies"
docs=docsearch.similarity_search(querey,k=3)



In [54]:
print("Result",docs)

Result [Document(metadata={'page': 130.0, 'source': 'data/Medical_book.pdf'}, page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(metadata={'page': 130.0, 'source': '../data/Medical_book.pdf'}, page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. T

In [40]:
prompt_template="""

Use the following pieces of information to answer the user's question. If you dont know the anser, just say that you don't know, do not try and piece together knowledge unless you are at least 80% sure that it is the correct answer. If you are less then 80% sure, try and point them in the direction of a good resource to find their answer.
Context: {context}
Question: {question}

Return the helpful answer and nothing else. 
Helpful answer:
"""


In [21]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}


In [22]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [23]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [24]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [25]:
model_path

'/Users/itamarlevi/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin'

In [26]:
llm = CTransformers(
    model='/Users/itamarlevi/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin',
    model_type="llama",
    config={'max_new_tokens': 512, 'temperature': 0.8},
)


In [33]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [34]:
retriever=docsearch.as_retriever(search_kwargs={'k': 2}),

In [35]:
%pip install langchain


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [36]:
from langchain.chains import combine_documents
print(dir(combine_documents))


['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'acollapse_docs', 'base', 'collapse_docs', 'create_stuff_documents_chain', 'map_reduce', 'map_rerank', 'reduce', 'refine', 'split_list_of_docs', 'stuff']


In [50]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain= create_stuff_documents_chain(llm,prompt_template)
retriever=docsearch.as_retriever(search_kwargs={'k': 2})


AttributeError: 'str' object has no attribute 'input_variables'

In [55]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)
